# Analysis of long-read transcriptomes

In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import anndata
import scipy.stats as st
import statsmodels.stats as stm
from statsmodels.stats.multitest import multipletests

### Data pre-processing

Run Minimap2, TranscriptClean, and TALON using `run_talon_tc.sh`

Create TALON abundance file
```bash 
db=talon/pgp1.db
talon_abundance \
    --db $db \
    -a gencode_v29 \
    -b hg38 \
    --o talon/pgp1
```

Filter novel transcripts for reproducibility
```bash
db=talon/pgp1.db
talon_filter_transcripts \
    --db $db \
    -a gencode_v29 \
    --maxFracA=0.5 \
    --minCount=5 \
    --minDatasets=2 \
    --o talon/pgp1_pass_list.csv
```

### TSS

Isolate reads that represent more confident 5' ends.

In [8]:
import pandas as pd
import swan_vis as swan

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [5]:
annot = 'talon/pgp1_talon_read_annot.tsv'

In [7]:
# limit the reads to those that represent putative 5' ends
df = pd.read_csv(annot, sep='\t')
tss_df = df.loc[df.transcript_novelty.isin(['Known', 'NIC', 'NNC', 'ISM'])]
tss_df = tss_df.loc[tss_df.ISM_subtype.isin(['None', 'Prefix', 'Both'])]
tss_reads = tss_df.read_name.tolist()

# tss
fname = 'tss_read_names.txt'
with open(fname, 'w') as ofile:
    for r in tss_reads:
        ofile.write(r+'\n')

Create a sam file with all the TSS reads from the merged BAM using picard tools `isolate_tss_reads.sh`

Call TSSs using Diane's script
```bash
tss_dir=~/mortazavi_lab/bin/tss-annotation/long_read/
python ${tss_dir}pacbio_to_tss.py \
    -i tss_reads.bam \
    --window-size=50 \
    --expression-threshold=2 \
    -o unfilt_tss.bed \
    -r \
    -n rev_tss.bw \
    -p fwd_tss.bw
```

Get the read names associated with each TSS as well 
```bash
python ${tss_dir}tss_reads.py \
    -i talon_tmp/merged.bam \
    -r unfilt_tss.bed \
    -o tss_reads.bed
```

In [325]:
# filter a list of TSSs for each gene
annot = 'talon/pgp1_talon_read_annot.tsv'
tss_reads = 'tss_reads.bed'

df = pd.read_csv(annot, sep='\t')

# remove sirvs and erccs
df = df.loc[~df.chrom.str.contains('SIRV')]
df = df.loc[~df.chrom.str.contains('ERCC')]

# print('Before assigning each read a TSS')
# print(len(df.index))
ends = pd.read_csv(tss_reads, sep='\t', header=None,
            names=['chrom', 'start', 'end', 'read_name', 'tss_id', 'stand'])

# merge on read name
df = df.merge(ends, how='inner', on='read_name')

# groupby on gene and tss
df = df[['read_name', 'annot_gene_name', 'tss_id']].groupby(['annot_gene_name', 'tss_id']).count()

# filter tsss for those that have >10% of the reads
# for the most highly-expressed tss of the gene
df.reset_index(inplace=True)
df.rename({'read_name':'count'}, axis=1, inplace=True)
df.loc[df.annot_gene_name == 'A1BG', 'count'].max()
temp = df.loc[df.apply(lambda x: x['count'] >= df.loc[df.annot_gene_name==x.annot_gene_name, 'count'].max()*0.1, axis=1)]

temp.loc[temp.annot_gene_name == 'MBP']

#, assign each TSS a name, and quantify TSS exp
temp.sort_values(by=['annot_gene_name'], inplace=True)
temp['tss_id_2'] = np.nan
prev_gene = None
for ind, entry in temp.iterrows():
    curr_gene = entry.annot_gene_name
    if curr_gene != prev_gene:
        i = 1
    else:
        i += 1
    prev_gene = curr_gene
    temp.loc[ind, 'tss_id_2'] = '{}_{}'.format(curr_gene, i)

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fairliereese/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/fairliereese/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-do

In [326]:
temp.head()

,annot_gene_name,tss_id,count,tss_id_2
0,A1BG,m54284U_210303_010717/56821818/ccs_11277,70,A1BG_1
1,A1BG-AS1,m54284_190620_121340/10027666/31_1705_CCS_11278,68,A1BG-AS1_1
2,A2M,m54284_190620_121340/62456306/4696_57_CCS_4485,43,A2M_1
4,A4GALT,m54284_190620_121340/71565601/31_2054_CCS_14051,38,A4GALT_1
5,AAAS,m54284_190612_161714/42402034/31_1873_CCS_4799,185,AAAS_1


In [327]:
temp.loc[temp.tss_id_2.str.endswith('_2')]

,annot_gene_name,tss_id,count,tss_id_2
9,AADAT,m54284_190610_233512/51970467/31_711_CCS_16063,11,AADAT_2
25,AASS,m54284_190613_123747/67633476/29_3042_CCS_19077,243,AASS_2
28,ABAT,m54284U_210304_072206/27723898/ccs_7420,26,ABAT_2
35,ABCB6,m54284_190618_193207/46662214/30_2343_CCS_12604,29,ABCB6_2
41,ABCC10,m54284_190618_193207/47841528/31_2258_CCS_17733,16,ABCC10_2
...,...,...,...,...
23319,ZSCAN12,m54284_190620_121340/73597637/3574_56_CCS_17396,13,ZSCAN12_2
23330,ZSCAN30,m54284U_210303_010717/132186743/ccs_9616,15,ZSCAN30_2
23333,ZSCAN31,m54284_190617_231034/11600701/42_1820_CCS_17393,4,ZSCAN31_2
23338,ZSCAN5A,m54284_190619_155256/40764222/29_1368_CCS_11212,10,ZSCAN5A_2


In [328]:
temp.loc[temp.tss_id_2.duplicated()]

,annot_gene_name,tss_id,count,tss_id_2


In [329]:
temp.loc[temp.annot_gene_name == 'AADAT']

,annot_gene_name,tss_id,count,tss_id_2
8,AADAT,m54284U_210304_072206/171836912/ccs_16062,13,AADAT_1
9,AADAT,m54284_190610_233512/51970467/31_711_CCS_16063,11,AADAT_2
10,AADAT,m54284_190612_161714/13894049/30_2117_CCS_16061,2,AADAT_3


In [330]:
# merged called TSSs back in with read annot
df = pd.read_csv(annot, sep='\t')
ends = pd.read_csv(tss_reads, sep='\t', header=None,
            names=['chrom', 'start', 'end', 'read_name', 'tss_id', 'stand'])
ends = ends[['read_name', 'tss_id']]

# merge on read name
df = df.merge(ends, how='inner', on='read_name')
temp = temp[['annot_gene_name', 'tss_id', 'tss_id_2']]
df = df.merge(temp, how='inner', on=['annot_gene_name', 'tss_id'])

# format like talon ab
cols = ['annot_gene_name', 'annot_gene_id', 'dataset', 'tss_id_2']
df = df[cols+['read_name']].groupby(cols).count().reset_index()
df.rename({'read_name':'counts', 'tss_id_2':'tss_id'}, axis=1, inplace=True)
df = df.pivot(index=['annot_gene_name', 'annot_gene_id', 'tss_id'], columns='dataset', values='counts')
df.reset_index(inplace=True)
df = df.rename_axis(None, axis=1)
df.fillna(0, inplace=True)

df.to_csv('pgp1_tss_talon_abundance.tsv', sep='\t', index=False)

In [331]:
df.head()

,annot_gene_name,annot_gene_id,tss_id,astro_1,astro_2,excite_neuron_1,excite_neuron_2,pgp1_1,pgp1_2
0,A1BG,ENSG00000121410.11,A1BG_1,8.0,3.0,18.0,11.0,22.0,8.0
1,A1BG-AS1,ENSG00000268895.5,A1BG-AS1_1,8.0,3.0,26.0,23.0,5.0,3.0
2,A2M,ENSG00000175899.14,A2M_1,2.0,1.0,11.0,29.0,0.0,0.0
3,A4GALT,ENSG00000128274.16,A4GALT_1,1.0,0.0,19.0,15.0,2.0,1.0
4,AAAS,ENSG00000094914.12,AAAS_1,16.0,6.0,20.0,29.0,53.0,61.0


In [333]:
print(len(df.loc[df.tss_id.duplicated(keep=False)].index))
df.loc[df.tss_id.duplicated(keep=False)]

26


,annot_gene_name,annot_gene_id,tss_id,astro_1,astro_2,excite_neuron_1,excite_neuron_2,pgp1_1,pgp1_2
4626,DIABLO,ENSG00000184047.17,DIABLO_1,63.0,31.0,68.0,85.0,106.0,79.0
4627,DIABLO,ENSG00000284934.1,DIABLO_1,1.0,0.0,2.0,0.0,3.0,1.0
4753,DNAJC9-AS1,ENSG00000227540.1,DNAJC9-AS1_1,0.0,0.0,4.0,1.0,5.0,0.0
4754,DNAJC9-AS1,ENSG00000236756.4,DNAJC9-AS1_1,0.0,0.0,1.0,3.0,0.0,2.0
7536,HSPA14,ENSG00000187522.15,HSPA14_1,17.0,9.0,13.0,13.0,67.0,49.0
7537,HSPA14,ENSG00000284024.2,HSPA14_1,9.0,10.0,2.0,4.0,2.0,5.0
10737,None,SIRV1.1,None_6,67.0,22.0,0.0,0.0,0.0,0.0
10738,None,SIRV1.2,None_6,94.0,49.0,0.0,0.0,0.0,0.0
10739,None,SIRV3.1,None_14,170.0,111.0,0.0,0.0,0.0,0.0
10740,None,SIRV3.2,None_14,60.0,18.0,0.0,0.0,0.0,0.0


### Swan

Swan config file

In [17]:
annot = '/Users/fairliereese/mortazavi_lab/ref/gencode.v29/gencode.v29.SIRV.ERCC.annotation.gtf'

# initialize SwanGraph
sg = swan.SwanGraph()

# add annotation GTF for reference
sg.add_annotation(annot)

# add transcript models from each dataset
sg.add_datasets('swan_config.tsv')

sg.save_graph('swan')


Adding dataset annotation to the SwanGraph

Adding dataset astro_1 to the SwanGraph

Adding dataset astro_2 to the SwanGraph

Adding dataset excite_neuron_1 to the SwanGraph

Adding dataset excite_neuron_2 to the SwanGraph

Adding dataset pgp1_1 to the SwanGraph

Adding dataset pgp1_2 to the SwanGraph
Saving graph as swan.p


### TSS switching

In [296]:
adata.var.head()

,gene_name,gene_id,tss_id
0,A1BG,ENSG00000121410.11,A1BG_1
1,A1BG-AS1,ENSG00000268895.5,A1BG-AS1_1
2,A1BG-AS1,ENSG00000268895.5,AC012313.2_1
3,A2M,ENSG00000175899.14,A2M_1
4,A4GALT,ENSG00000128274.16,A4GALT_1


In [308]:
# df: talon abundance file (either tss or transcript)
# cond_map: dictionary of {condition: [dataset1, dataset2]}; how you want to group datasets
# how: whether to make a tss or iso level adata; 'tss' or 'iso'
# pass_list: if 'iso', file of valid transcript IDs that pass filtering
def make_adata(df, cond_map, how='iso', pass_list=None):
    
    # filter talon ab file based on pass list
#     df = pd.read_csv(ab_file, sep='\t')
    if pass_list:
        pass_list = pd.read_csv(pass_list, header=None, names=['gene_id', 'transcript_id'])
        df = df.loc[df.transcript_ID.isin(pass_list.transcript_id.tolist())]

    # obs table
    obs = pd.DataFrame.from_dict(cond_map, orient='index')
    obs.reset_index(inplace=True)
    id_vars = ['index']
    value_vars = obs.columns[1:]
    obs = obs.melt(id_vars=id_vars, value_vars=value_vars)
    obs.drop('variable', axis=1, inplace=True)
    obs.rename({'index':'condition', 'value':'dataset'}, axis=1, inplace=True)

    # var table
    if how=='iso':
        var_cols = ['annot_transcript_name', 'annot_gene_name',\
                    'annot_transcript_id', 'annot_gene_id', \
                    'gene_ID', 'transcript_ID', 'transcript_novelty', \
                    'ISM_subtype']
        var = df[var_cols]
        var.rename({'transcript_ID':'transcript_id', \
                    'gene_ID':'gene_id',\
                    'annot_gene_name': 'gene_name'}, axis=1, inplace=True)
    if how=='tss': 
        var_cols = ['annot_gene_name', 'annot_gene_id', 'tss_id']
        var = df[var_cols]
        var.rename({'annot_gene_name': 'gene_name',\
                    'annot_gene_id':'gene_id'}, axis=1, inplace=True)
        
    # X table
    df = df.transpose()
    df = df.loc[df.index.isin(obs.dataset.tolist())]
    obs_order = obs['dataset'].reset_index().set_index('dataset')
    df['dataset_num'] = df.index.map(obs_order['index'])
    df.sort_values('dataset_num', inplace=True)
    df.drop('dataset_num', axis=1, inplace=True)
    X = df.to_numpy()

    adata = anndata.AnnData(obs=obs, var=var, X=X) 
    
    return adata

# gene_df: pandas dataframe with expression values in each condition for each TSS in a gene
# conditions: list of str of condition names
# rc: threshold of read count per gene in each condition necessary to test this gene
def test_gene(gene_df, conditions, col, id_col, rc=10):
    
    gene_df = gene_df.pivot(index=col, columns=id_col, values='counts')
    gene_df = gene_df.transpose()
    
    groups = gene_df.columns.tolist()
    gene_df['total_counts'] = gene_df[groups].sum(axis=1)
    gene_df.sort_values(by='total_counts', ascending=False, inplace=True)

    if len(gene_df.index) > 11:
        gene_df.reset_index(inplace=True)

        beep = gene_df.iloc[10:].sum()
        beep[id_col] = 'all_other'
        beep.index.name = None  
        beep = pd.DataFrame(beep).transpose()

        gene_df = gene_df.iloc[:10]
        gene_df = pd.concat([gene_df, beep])  
        
    # limit to just isoforms with > 0 expression in both conditions
    cond1 = conditions[0]
    cond2 = conditions[1]
    gene_df = gene_df.loc[(gene_df[cond1]>0)&(gene_df[cond2]>0)]
    
    # does this gene reach the desired read count threshold?
    for cond in conditions:
        if gene_df[cond].sum() < rc:
            return np.nan, np.nan
    
    # only do the rest if there's nothing left
    if gene_df.empty:
        return np.nan, np.nan
    
    # calculate the percent of each sample each TSS accounts for
    cond_pis = []
    for cond in conditions:
        total_col = '{}_total'.format(cond)
        pi_col = '{}_pi'.format(cond)
        total_count = gene_df[cond].sum()

        cond_pis.append(pi_col)

        gene_df[total_col] = total_count
        gene_df[pi_col] = (gene_df[cond]/gene_df[total_col])*100
        
    # compute isoform-level and gene-level delta pis
    gene_df['dpi'] = gene_df[cond_pis[0]] - gene_df[cond_pis[1]]
    gene_df['abs_dpi'] = gene_df.dpi.abs()
    gene_dpi = gene_df.iloc[:2].abs_dpi.sum()    
    
    # chi squared test 
    chi_table = gene_df[conditions].to_numpy()
    chi2, p, dof, exp = st.chi2_contingency(chi_table)
    
    return p, gene_dpi

def filter_die_results(df, p, dpi):
    df = df.loc[(df.adj_p_val<=p)&(df.dpi>=dpi)]
    return df

# adata: adata with TSS or iso expression 
# conditions: len 2 list of strings of conditions to compare
# col: string, which column the condition labels are in
# how: 'tss' or 'iso'
def get_die(adata, conditions, how='tss', rc=15):
    
    if how == 'tss':
        id_col = 'tss_id'
    elif how == 'iso':
        id_col = 'transcript_id'
    
    # make df that we can groupby
    col = 'condition'
    colnames = adata.var[id_col].tolist()
    print(len(colnames))
    print(len(set(colnames)))
    rownames = adata.obs.dataset.tolist()    
    raw = adata.X
    df = pd.DataFrame(data=raw, index=rownames, columns=colnames)
    df.reset_index(inplace=True)
    df.rename({'index':'dataset'}, axis=1, inplace=True)
    samp = adata.obs[['dataset', col]]
    df = df.merge(samp, how='left', on='dataset')
    
    # limit to only the samples that we want in this condition
    df[col] = df[col].astype('str')
    df = df.loc[df[col].isin(conditions)]
        
    # groupby sample type and sum over gen
    df.drop('dataset', axis=1, inplace=True)
    df = df.groupby(col).sum().reset_index()
    
    # melty boi
    tss_cols = df.columns.tolist()[1:]
    print(tss_cols[:5])
    print(df.head())
    df = df.melt(id_vars=col, value_vars=tss_cols)
    
    # rename some cols
    df.rename({'variable':id_col,'value':'counts'}, axis=1, inplace=True)
    
    # merge with gene names
    df = df.merge(adata.var, how='left', on=id_col)
    
    # get total number of tss or iso / gene
    bop = df[['gene_name', id_col]].groupby('gene_name').count().reset_index()
    
    # construct tables for each gene and test!
    gene_names = df.gene_name.unique().tolist()
    gene_de_df = pd.DataFrame(index=gene_names, columns=['p_val', 'dpi'], data=[[np.nan for i in range(2)] for j in range(len(gene_names))])
    for gene in gene_names:
        gene_df = df.loc[df.gene_name==gene]
        p, dpi = test_gene(gene_df, conditions, col, id_col, rc=rc)
        gene_de_df.loc[gene, 'p_val'] = p
        gene_de_df.loc[gene, 'dpi'] = dpi
        
    # correct p values 
    gene_de_df.dropna(axis=0, inplace=True)
    p_vals = gene_de_df.p_val.tolist()
    _, adj_p_vals, _, _ = multipletests(p_vals, method='fdr_bh')
    gene_de_df['adj_p_val'] = adj_p_vals
    
        
    return gene_de_df

In [309]:
ab_file = 'talon/pgp1_tss_talon_abundance.tsv'
cond_map = {'Astrocytes': ['astro_1', 'astro_2'], \
            'Excitatory neurons': ['excite_neuron_1', 'excite_neuron_2'], \
            'PGP1': ['pgp1_1', 'pgp1_2']}

df = pd.read_csv(ab_file, sep='\t')
adata = make_adata(df, cond_map, how='tss')

# do one test for each pair of conditions
tested = []
conditions = ['Astrocytes', 'Excitatory neurons', 'PGP1']

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
Transforming to str index.
Transforming to str index.


In [310]:
how = 'tss'
for c1 in conditions:
    for c2 in conditions: 
        if (c1, c2) in tested or c1 == c2 or (c2, c1) in tested:
            continue
        else:
            tested.append((c1,c2))
        df = get_die(adata, [c1, c2], how=how, rc=10)
        fname = '{}_{}_{}_die.tsv'.format(c1, c2, how)
        df.to_csv(fname, sep='\t', index=False)

20997
18064
['A1BG_1', 'A1BG-AS1_1', 'AC012313.2_1', 'A2M_1', 'A4GALT_1']
            condition  A1BG_1  A1BG-AS1_1  AC012313.2_1  A2M_1  A4GALT_1  \
0          Astrocytes    11.0        11.0          11.0    3.0       1.0   
1  Excitatory neurons    29.0        49.0          49.0   40.0      34.0   

   AAAS_1  AACS_1  AADAT_1  AADAT_2  ...  ZSWIM7_2  ZSWIM8_1  ZUP1_1  ZW10_1  \
0    22.0    54.0      1.0      8.0  ...       3.0       2.0    32.0    20.0   
1    49.0    28.0      0.0      0.0  ...      25.0       0.0    35.0     9.0   

   ZWILCH_1  ZWINT_1  ZXDC_1  ZYG11B_1  ZYX_1  ZZEF1_1  
0      42.0     18.0    16.0      12.0  543.0      3.0  
1      22.0     50.0     2.0       8.0   56.0      5.0  

[2 rows x 20998 columns]


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

### Isoform switching

In [303]:
pass_list = 'talon/pgp1_pass_list.csv'
ab_file = 'talon/pgp1_talon_abundance.tsv'
cond_map = {'Astrocytes': ['astro_1', 'astro_2'], \
            'Excitatory neurons': ['excite_neuron_1', 'excite_neuron_2'], \
            'PGP1': ['pgp1_1', 'pgp1_2']}

df = pd.read_csv(ab_file, sep='\t')
adata = make_adata(df, cond_map, pass_list=pass_list)

/Users/fairliereese/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
Transforming to str index.
Transforming to str index.


In [305]:
adata.var.head()

,annot_transcript_name,gene_name,annot_transcript_id,annot_gene_id,gene_id,transcript_id,transcript_novelty,ISM_subtype
0,MIR1302-2HG-201,MIR1302-2HG,ENST00000469289.1,ENSG00000243485.5,4,6,Known,None
1,FAM138A-201,FAM138A,ENST00000417324.1,ENSG00000237613.2,6,8,Known,None
2,FAM138A-202,FAM138A,ENST00000461467.1,ENSG00000237613.2,6,9,Known,None
3,CICP27-201,CICP27,ENST00000442987.3,ENSG00000233750.3,12,21,Known,None
4,AL627309.6-201,AL627309.6,ENST00000494149.2,ENSG00000268903.1,13,22,Known,None


In [304]:
# do one test for each pair of conditions
tested = []
conditions = ['Astrocytes', 'Excitatory neurons', 'PGP1']

how = 'iso'
for c1 in conditions:
    for c2 in conditions: 
        if (c1, c2) in tested or c1 == c2 or (c2, c1) in tested:
            continue
        else:
            tested.append((c1,c2))
        df = get_die(adata, [c1, c2], how=how, rc=10)
        fname = '{}_{}_{}_die.tsv'.format(c1, c2, how)
        df.to_csv(fname, sep='\t', index=False)

[6, 8, 9, 21, 22]
            condition    6    8    9   21   22   32  132  136  138  ...  \
0          Astrocytes  0.0  0.0  0.0  1.0  1.0  7.0  1.0  4.0  0.0  ...   
1  Excitatory neurons  1.0  0.0  0.0  0.0  3.0  7.0  2.0  3.0  5.0  ...   

   497872  497898  498029  498062  498451  498466  499084  499092  499187  \
0     4.0     1.0    20.0    12.0     0.0    11.0     0.0     0.0     9.0   
1    25.0    20.0     1.0     8.0     9.0     2.0    10.0    20.0   123.0   

   499325  
0    19.0  
1     7.0  

[2 rows x 40232 columns]


KeyboardInterrupt: 

In [172]:
adata.var.head()

,annot_transcript_id,annot_gene_id,gene_ID,transcript_ID,transcript_novelty,ISM_subtype
0,ENST00000469289.1,ENSG00000243485.5,4,6,Known,None
1,ENST00000417324.1,ENSG00000237613.2,6,8,Known,None
2,ENST00000461467.1,ENSG00000237613.2,6,9,Known,None
3,ENST00000442987.3,ENSG00000233750.3,12,21,Known,None
4,ENST00000494149.2,ENSG00000268903.1,13,22,Known,None
